*This is an initial look into paths of classifying monet paintings. Due to current status may be worth even trying to do more genre based vs. artist based look into the situation.* 
  
*The created neural network used keras on top of a theano backend. Currently using a three stage network to categorize the images.*

In [1]:
import theano

from keras.models import Sequential        # Linear stack


from keras.layers import (                 
    # Flattened list of layers
    
                          Convolution2D,   
                            # Filters via windows of 2d input
                          MaxPooling2D,    
                            # Subsets image into matrices, largest value of matrix is taken
                          Activation,      
                            # Filters tha activate when deisred features are identified
                          Dropout,         
                            # Prevents overfitting by dropping based on probability 1-p
                          Flatten,         
                            # Layers comprising model graph
                          Dense            
                            # Specify input argument shape
    
                          )

Using Theano backend.


*In order to start it is important to begin by prcessing the data. This is a quick look into how it's done, will do quick demo*   
  
*Basically the image that will be looked at is taken, processed in a number of different ways (flipped, skewed, cropped, etc.). This is like turning a signature upside down for forgery. Looking at the situation through several angles.*

In [2]:
# Data Cleaning and Preparation

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



d_gen = ImageDataGenerator( featurewise_center = False, # input mean as 0 over data
                            samplewise_center = False,  # Set sample mean to 0
                            featurewise_std_normalization=False, # Standardize around std of dataset 
                            samplewise_std_normalization=False,  # Standardize around std of sample
                            zca_whitening=False, # Apply ZCA whitening
                           
                            rotation_range=0.15,   # Random Roation within specified range
                            width_shift_range=0.15, # Horizontal shift as a fractional of image width
                            height_shift_range=0.15, # Vertical shift as fraction of image height
                            shear_range=0.15, # Intensity of shear
                            zoom_range=0.15, # Either number or list, but specifies the range for the random zoom
                            
                            channel_shift_range=0., # Range for random channel shifts
                            fill_mode='nearest', # How pixels outside of the boundary are dealt with
                            cval=0., # color value for fill_mode if that value was set to 'constant'
                            horizontal_flip=True, # Random horizontal flip of sample
                            vertical_flip=True, # Random vertical flip of sample
                            rescale=1./225, # Scalar multiplying the image. 1 used  
#                             dim_ordering=K.image_dim_ordering()
                          )

In [3]:
monet1 = load_img('Claude_Monet,_Impression,_soleil_levant.jpg') # PIL image
monet_array = img_to_array(monet1) # Convert the 2d Image to an array
monet_array = monet_array.reshape((1,) + monet_array.shape) # Reshape to 1D

Show the purdy pictures.

In [4]:
t = 0
for batch in d_gen.flow(monet_array, batch_size=1,
                        save_to_dir='mod', save_prefix='monet', 
                        save_format='jpeg'):
    t += 1
    if t > 20: # Create 20 manipulated images
        break 

# Creation of the Model

*Here is where the model is created. So far the model that is being used is a rather simple three step neural network. Then going to build out a bit more*

In [5]:

model = Sequential()
model.add(Convolution2D(32,3,3, input_shape=(3,150,150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(32,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Build the model

In [6]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))           # Rectified Linear Unit
model.add(Dropout(0.5))                 # Drop 50% of the Neurons
model.add(Dense(1))
model.add(Activation('sigmoid'))        # Yay, sigmoids

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
# use the above datagen for training
d_gen = ImageDataGenerator(rescale=1./255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          horizontal_flip=True)

# For test only use a rescale function
test_d_gen = ImageDataGenerator(rescale=1./255)

In [24]:
monet_gen = d_gen.flow_from_directory('/Users/sebozek/Desktop/Test/',       # Folder containing the test images
                                     target_size = (150, 150),              # Resize to 150 * 150 pxs
                                     batch_size = 32,                       # Reduce Noise, larger batch less noise
                                     class_mode='binary')                   # Binary Classification Problem

hold_out = test_d_gen.flow_from_directory('/Users/sebozek/Desktop/Test/',
                                         target_size= (150, 150),
                                         batch_size = 32,
                                         class_mode='binary')

Found 3931 images belonging to 2 classes.
Found 3931 images belonging to 2 classes.


In [25]:
model.fit_generator(monet_gen,                  # Resize the image get from test dictionary
                   samples_per_epoch=800,      # How many amples to use per epoch
                   nb_epoch=25,                 # Number of epochs used
                   validation_data = hold_out,  # Location of the validation data
                   nb_val_samples=1200,          # Validation samples used per epoch
                   verbose=1)

model.save_weights('monet_weights.h5')
# save as JSON
json_string = model.to_json('monet_model.json')

Epoch 1/25
800/800 [==============================] - 175s - loss: 0.3277 - acc: 0.8862 - val_loss: 0.2002 - val_acc: 0.9095
Epoch 2/25
800/800 [==============================] - 148s - loss: 0.2220 - acc: 0.9225 - val_loss: 0.3823 - val_acc: 0.8125
Epoch 3/25
800/800 [==============================] - 157s - loss: 0.2511 - acc: 0.9187 - val_loss: 0.1497 - val_acc: 0.9474
Epoch 4/25
800/800 [==============================] - 188s - loss: 0.1993 - acc: 0.9125 - val_loss: 0.1502 - val_acc: 0.9521
Epoch 5/25
827/800 [===============================] - 222s - loss: 0.2432 - acc: 0.8888 - val_loss: 0.1663 - val_acc: 0.9581
Epoch 6/25
800/800 [==============================] - 172s - loss: 0.2221 - acc: 0.9125 - val_loss: 0.1224 - val_acc: 0.9556
Epoch 7/25
800/800 [==============================] - 224s - loss: 0.2506 - acc: 0.9100 - val_loss: 0.1788 - val_acc: 0.9149
Epoch 8/25
800/800 [==============================] - 263s - loss: 0.2484 - acc: 0.8975 - val_loss: 0.1447 - val_acc: 0.9507

KeyboardInterrupt: 

* Test : 
    * Monet:
    * Not Monet:
* Hold Out: **No Issues**
    * Monet:
    * Not Monet:

*Initial running of the model found there was an issue with the image files and was given this error message:  
**IOError: image file is truncated (39 bytes not processed)**  
From [StackOverflow](http://bit.ly/21oXdhF) it was found that the images were likely corrupted or had unneeded trailing bytes. In order to find these truncated files used [jpeginfo](https://github.com/tjko/jpeginfo) to create a text file of the analysis of the jpegs. Output read by pandas allows for isolation of the trouble data.*


In [17]:
import pandas as pd
import numpy as np

In [19]:
test_monet = pd.read_csv("/Users/sebozek/Desktop/Test.txt",delim_whitespace=True,header=None)
test_not_monet = pd.read_csv("/Users/sebozek/Desktop/Test.txt",delim_whitespace=True,header=None)
val_monet = pd.read_csv("/Users/sebozek/Desktop/HoldOut.txt",delim_whitespace=True,header=None, error_bad_lines=False)
val_not_monet = pd.read_csv("/Users/sebozek/Desktop/HoldOut_nm.txt", delim_whitespace=True, header=None,  error_bad_lines=False)

CParserError: Error tokenizing data. C error: Expected 15 fields in line 304, saw 16


In [ ]:
dfs = [test_monet, test_not_monet, val_monet, val_not_monet]

# More Descriptive Labels
for df in dfs:
    df.columns = ["img_name", "width", "x", 'height', 'bits', 'format', 'N/P', 'pixels', 'test']

In [ ]:
for df in dfs:
    print df.test.unique()
    
print val_not_monet.test.unique()

In [ ]:
val_not_monet['img_name'][val_not_monet.test == '[ERROR]'].values

## To Do

* Expand to try categorizing styles?
* Figure out a few hatches
* Try getting to above a 95% accuracy
* Manipulate loss functions
* Change the dropout rate and see what's going on
* Validation
    * ROC curve
    * Confusion Matrix
* What does model think is important?
* Restricted Boltzmann Machine for Monet Replication

In [ ]:
# Import previous model (that worked)
model.load_weights('my_model_weights.h5')

# Get image output
output_1 = K.function([model.layers[0].input],
                     [model.layers[1].output])

In [ ]:
test_not_monet.head(1)